<a href="https://colab.research.google.com/github/leoquiroa/TensorFlow/blob/master/site/en/tutorials/quickstart/beginner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Load a prebuilt dataset.
1. Build a neural network machine learning model that classifies images.
2. Train this neural network.
3. Evaluate the accuracy of the model.

This tutorial is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

## Set up TensorFlow

Import TensorFlow into your program to get started:

In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.8.0


If you are following along in your own development environment, rather than [Colab](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb), see the [install guide](https://www.tensorflow.org/install) for setting up TensorFlow for development.

Note: Make sure you have upgraded to the latest `pip` to install the TensorFlow 2 package if you are using your own development environment. See the [install guide](https://www.tensorflow.org/install) for details.

## Load a dataset

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the sample data from integers to floating-point numbers:

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11501568/11490434 [==============================] - 0s 0us/step


In [4]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


## Build a machine learning model

Build a `tf.keras.Sequential` model by stacking layers.

In [66]:
all_layers = tf.keras.layers
dummy_model = tf.keras.models.Sequential([
  all_layers.Flatten(input_shape=(28, 28)),
  all_layers.Dense(128, activation='relu'),
  all_layers.Dropout(0.2),
  all_layers.Dense(10)
])

In [65]:
real_model = dummy_model

For each example, the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

## Prediction

In [37]:
x_train[:1].shape

(1, 28, 28)

Almost random prediction

In [62]:
predictions = dummy_model(x_train[:1]).numpy()
predictions

array([[ 0.2028989 ,  0.08862238, -0.2905482 ,  0.43096983, -0.35039717,
         0.38743228, -0.787111  , -0.43454596, -0.26439196,  0.29274058]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to *probabilities* for each class: 

In [22]:
tf.nn.softmax(predictions).numpy()

array([[0.1176527 , 0.1364746 , 0.08397663, 0.09166975, 0.05969913,
        0.1476696 , 0.1224132 , 0.07193227, 0.07373379, 0.09477839]],
      dtype=float32)

Note: It is possible to bake the `tf.nn.softmax` function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output. 

## Define a loss function

Define a loss function for training using `losses.SparseCategoricalCrossentropy`, which takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [23]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [24]:
y_train[:1]

array([5], dtype=uint8)

In [25]:
predictions

array([[ 0.64066905,  0.7890705 ,  0.3034705 ,  0.39112443, -0.03775066,
         0.8679093 ,  0.6803342 ,  0.14865701,  0.17339312,  0.42447338]],
      dtype=float32)

In [26]:
loss_fn(y_train[:1], predictions).numpy()

1.912778

## Compile

Before you start training, configure and compile the model using Keras `Model.compile`. Set the [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [63]:
real_model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Train and evaluate your model

Use the `Model.fit` method to adjust your model parameters and minimize the loss: 

In [64]:
real_model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2939 - accuracy: 0.9148
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1437 - accuracy: 0.9573
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1083 - accuracy: 0.9672
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0887 - accuracy: 0.9729
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0758 - accuracy: 0.9765


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [67]:
real_model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0783 - accuracy: 0.9755 - 517ms/epoch - 2ms/step


[0.07831291854381561, 0.9754999876022339]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [68]:
probability_model = tf.keras.Sequential([
  real_model,
  tf.keras.layers.Softmax()
])

The probability of the first five elements for each of the ten possible outcomes

In [69]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[4.6594934e-08, 1.9634663e-09, 2.5388979e-06, 1.3507964e-04,
        3.6350634e-13, 2.3414184e-07, 2.4103367e-13, 9.9985600e-01,
        2.1452853e-07, 5.8529995e-06],
       [1.6765865e-05, 1.7266568e-04, 9.9767739e-01, 1.8035305e-03,
        2.4544115e-13, 1.1081979e-04, 2.4076468e-05, 2.0038086e-10,
        1.9484269e-04, 1.6120230e-10],
       [2.8029044e-07, 9.9840385e-01, 1.2749432e-04, 4.9344650e-05,
        8.4096078e-05, 6.7024928e-05, 8.4674248e-06, 4.5884566e-04,
        7.9937797e-04, 1.2280423e-06],
       [9.9960023e-01, 7.2239176e-10, 8.5052425e-06, 4.5351879e-07,
        4.0682289e-09, 1.9543400e-05, 4.5061952e-05, 3.0342577e-04,
        1.0323467e-07, 2.2692742e-05],
       [1.1262008e-06, 3.0948375e-09, 1.3094825e-05, 2.5888343e-07,
        9.9592304e-01, 3.7145102e-07, 1.0656921e-06, 4.6313307e-05,
        2.0488800e-07, 4.0146816e-03]], dtype=float32)>

## Conclusion

Congratulations! You have trained a machine learning model using a prebuilt dataset using the [Keras](https://www.tensorflow.org/guide/keras/overview) API.

For more examples of using Keras, check out the [tutorials](https://www.tensorflow.org/tutorials/keras/). To learn more about building models with Keras, read the [guides](https://www.tensorflow.org/guide/keras). If you want learn more about loading and preparing data, see the tutorials on [image data loading](https://www.tensorflow.org/tutorials/load_data/images) or [CSV data loading](https://www.tensorflow.org/tutorials/load_data/csv).
